In [1]:
#!kaggle datasets download -d fareselmenshawii/face-detection-dataset -p data --unzip

In [2]:
from ultralytics import YOLO

import torch

In [3]:
device = 'mps' if torch.mps.is_available() else 'cpu'

In [4]:
model = YOLO('yolo11n.pt')

In [5]:
import os

# Проверяем структуру
print("=== Проверка путей ===")
print(f"Train images: {len(os.listdir('../data/images/train'))} файлов")
print(f"Val images: {len(os.listdir('../data/images/val'))} файлов")
print(f"labels2: {len(os.listdir('../data/labels2'))} файлов")

=== Проверка путей ===
Train images: 13386 файлов
Val images: 3347 файлов
labels2: 3 файлов


In [6]:
import os
import shutil

# === Пути ===
data_root = '../data'
images_train = os.path.join(data_root, 'images/train')
images_val = os.path.join(data_root, 'images/val')
labels_src = os.path.join(data_root, 'labels2')
labels_train = os.path.join(data_root, 'labels2/train')
labels_val = os.path.join(data_root, 'labels2/val')

# === 1. Создаём папки для меток ===
os.makedirs(labels_train, exist_ok=True)
os.makedirs(labels_val, exist_ok=True)

# === 2. Функция распределения меток ===
def distribute_labels(image_folder, label_dest_folder):
    moved = 0
    for img_name in os.listdir(image_folder):
        if not img_name.lower().endswith(('.jpg', '.png', '.jpeg')):
            continue
        
        # Получаем имя без расширения
        base_name = os.path.splitext(img_name)[0]
        label_file = f"{base_name}.txt"
        
        src = os.path.join(labels_src, label_file)
        dst = os.path.join(label_dest_folder, label_file)
        
        if os.path.exists(src):
            shutil.move(src, dst)
            moved += 1
    
    return moved

# === 3. Запуск ===
print("🔄 Распределение меток...")
train_count = distribute_labels(images_train, labels_train)
val_count = distribute_labels(images_val, labels_val)

print(f"✅ Train меток: {train_count}")
print(f"✅ Val меток: {val_count}")

# === 4. Проверка ===
print("\n📊 Итоговая структура:")
print(f"   images/train: {len(os.listdir(images_train))} файлов")
print(f"   images/val: {len(os.listdir(images_val))} файлов")
print(f"   labels2/train: {len(os.listdir(labels_train))} файлов")
print(f"   labels2/val: {len(os.listdir(labels_val))} файлов")

🔄 Распределение меток...
✅ Train меток: 0
✅ Val меток: 0

📊 Итоговая структура:
   images/train: 13386 файлов
   images/val: 3347 файлов
   labels2/train: 13386 файлов
   labels2/val: 3347 файлов


In [ ]:
# Отключаем wandb и другие внешние логи
os.environ['WANDB_MODE'] = 'disabled'
os.environ['COMET_MODE'] = 'disabled'
os.environ['MLFLOW_TRACKING_URI'] = 'disabled'

results = model.train(
    data='../data/data.yaml',
    epochs=30,
    imgsz=320,      # ⚠️ Совет: уменьши с 640 до 320 для 8 ГБ ОЗУ
    batch=4,
    device='mps',
    workers=2,
    cache=False,
    amp=True,
    exist_ok=True,
    # Отключаем внешние логи:
    project=None,   # Не создавать проект на сервере
    name='face_detection'
)